<a href="https://colab.research.google.com/github/jwilsonschutter/Assignment-4-Webmapping/blob/main/littlemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Here I install all of the packages I'll be using today! (some were used )

import pandas as pd
%pip install censusdata 
import censusdata as cd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
import statsmodels.formula.api as sm
import numpy as np
%pip install --upgrade --quiet plotly
import plotly.express as px
import requests
import json

     |████████████████████████████████| 26.6 MB 73 kB/s 
  Created wheel for censusdata: filename=CensusData-1.15-py3-none-any.whl size=28205534 sha256=3755a734c7ac154e988e9057600942c3a866b8769bdd04b2287530ee00b47940
  Stored in directory: /root/.cache/pip/wheels/17/11/8c/933901298f486bd414f2ab1a62a114085f7d7a19dcbda2dd08
Successfully built censusdata


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 23.9 MB 14 kB/s 


We'll start by looking at the 2010 5 year ACS, where we see that the most affluent part of Manhattan pulls in an average houshold income of over 230 thousand dollars a year, and this is around the upper east side, represented by census tract 130. The following area is also right next door, and is census tract 142 pulling in around 10k less annually than it's neighbor. The rest of the most affluent areas are located nearby, or in the most southern tip of Manhattan near the Financial District. We also see that the worst off areas of Manhattan are pulling in less than 10k annually, and this would be right around Midtown, with other lower-income areas to the north of central park around Harlem, or on the south-eastern banks of NYC's Lower East Side.

In [ ]:
#download the data from the census and turn it into a data frame for 2019, needed to use acs 5 as acs 1 didn't have the tract level, only county!
newyorkcounty2010 = cd.download('acs5', 2010, cd.censusgeo([('state', '36'), ('county', '061'), ('tract', '*')]),['B19013_001E'])

df_2010 = pd.DataFrame(newyorkcounty2010)
#make the index a column in it's own right so it's easier to deal with
df_2010_full = df_2010.reset_index()
df_2010_full = df_2010_full.rename(columns = {'index':'census_tract'})
df_2010_full = df_2010_full.rename(columns = {'B19013_001E':'hh_median_income'})
df_2010_full['hh_median_income'] = df_2010_full['hh_median_income'].astype(int)
df_2010_full['census_tract'] = df_2010_full['census_tract'].astype(str)
df_2010_full['tracts_simplified'] = df_2010_full['census_tract'].str[-6:]
df_2010_full['temp_additative'] = 1
df_2010_full['temp_additative'] = df_2010_full['temp_additative'].astype(str)
df_2010_full['final_tracts'] = df_2010_full['temp_additative'] + df_2010_full['tracts_simplified']
#the following two lines are done to drop columns that don't have any value as some places like central park or other public properties doen't have people who live there!
df_2010_full = df_2010_full[df_2010_full['hh_median_income'].notna()]
df_2010_full = df_2010_full[df_2010_full['hh_median_income'] > 0]
#now for the display
df_2010_display = df_2010_full[['census_tract','hh_median_income']]
df_2010_display = df_2010_display.sort_values(by=['hh_median_income'], ascending=False)
df_2010_display = df_2010_display.rename(columns= {'census_tract':'Census Tract', 'hh_median_income':'Median Household Income'})
df_2010_full = df_2010_full.rename(columns= {'hh_median_income':'Median Household Income'})
df_2010_display.head()



,Census Tract,Median Household Income
244,"Census Tract 130, New York County, New York: S...",232768
255,"Census Tract 142, New York County, New York: S...",222829
133,"Census Tract 21, New York County, New York: Su...",203849
279,"Census Tract 160.01, New York County, New York...",189936
267,"Census Tract 150.02, New York County, New York...",170486


In [ ]:
newyorkcounty2010 = cd.download('acs5', 2010, cd.censusgeo([('state', '36'), ('county', '061'), ('tract', '*')]),['B19013_001E'])


,Census Tract,Median Household Income
79,"Census Tract 243.02, New York County, New York...",16505
137,"Census Tract 25, New York County, New York: Su...",16299
24,"Census Tract 192, New York County, New York: S...",16157
234,"Census Tract 119, New York County, New York: S...",13856
207,"Census Tract 94, New York County, New York: Su...",9750


When examining the end of the decade, or the 2019 5 year ACS, we see that the average household income has increased dramatically in the most affluent parts of Manhattan, hitting over the maximum reportable data by the ACS, i.e. over 250k annually, and similarly to in 2010, the most affluent census tracts seem to be around the upper east side and the financial district! We see that poverty seems to be less prounced with even the least fiscally gifted tracts still earning over 5k more than the previous lowest, meaning that at the very least everyone in Manhattan has gotten richer over the previous decade. Similarly, Harlem, and the lowest east side, along with certain parts of Midtown take the cake here for the lowest earning census tracts.

In [ ]:
#download the data from the census and turn it into a data frame for 2019 061 manhattan, 047 kings / brooklyn
newyorkcounty2019 = cd.download('acs5', 2019, cd.censusgeo([('state', '36'), ('county', '047'), ('tract', '*')]),['B19013_001E'])
df_2019 = pd.DataFrame(newyorkcounty2019)
#make the index a column in it's own right so it's easier to deal with
df_2019_full = df_2019.reset_index()
df_2019_full = df_2019_full.rename(columns = {'index':'census_tract'})
df_2019_full = df_2019_full.rename(columns = {'B19013_001E':'hh_median_income'})
df_2019_full['hh_median_income'] = df_2019_full['hh_median_income'].astype(int)
df_2019_full['census_tract'] = df_2019_full['census_tract'].astype(str)
df_2019_full['tracts_simplified'] = df_2019_full['census_tract'].str[-6:]
df_2019_full['temp_additative'] = 1
df_2019_full['temp_additative'] = df_2019_full['temp_additative'].astype(str)
df_2019_full['final_tracts'] = df_2019_full['temp_additative'] + df_2019_full['tracts_simplified']
#the following two lines are done to drop columns that don't have any value as some places like central park or other public properties doen't have people who live there!
df_2019_full = df_2019_full[df_2019_full['hh_median_income'].notna()]
df_2019_full = df_2019_full[df_2019_full['hh_median_income'] > 0]
#now we're on to the display
df_2019_display = df_2019_full[['census_tract','hh_median_income']]
df_2019_display = df_2019_display.sort_values(by=['hh_median_income'], ascending=False)
df_2019_display = df_2019_display.rename(columns= {'census_tract':'Census Tract','hh_median_income':'Median Household Income'})
df_2019_full = df_2019_full.rename(columns= {'hh_median_income':'Median Household Income'})
df_2019_display.head()

,Census Tract,Median Household Income
313,"Census Tract 21, Kings County, New York: Summa...",228276
685,"Census Tract 133, Kings County, New York: Summ...",192813
191,"Census Tract 165, Kings County, New York: Summ...",173939
584,"Census Tract 167, Kings County, New York: Summ...",173113
681,"Census Tract 67, Kings County, New York: Summa...",171477


In [ ]:
brooklyn2019race = cd.download('acs5', 2019, cd.censusgeo([('state', '36'), ('county', '047'), ('tract', '*')]),['DP05_0038PE'])
df_race_2019 = pd.DataFrame(brooklyn2019race)
df_race_2019.head()

ValueError: ignored

In [ ]:
geojson = 'https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Census_Tracts_for_2010_US_Census/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson'
response = requests.get(geojson)
response.json()

fig_mapbox_2010 = px.choropleth_mapbox(df_2010_full,
                                  geojson=geojson,
                                  locations='final_tracts',
                                  featureidkey='properties.BoroCT2010',
                                  color='Median Household Income',
                                  color_continuous_scale="Viridis",
                                  range_color=(0, 250005),
                                  hover_data=['Median Household Income'],
                                  center = {'lat': 40.79, 'lon': -73.99},
                                  zoom=10.5,
                                  mapbox_style='carto-positron')

fig_mapbox_2010.update_layout(height=700)
fig_mapbox_2010.show()

In [ ]:
fig_mapbox_2019 = px.choropleth_mapbox(df_2019_full,
                                  geojson=geojson,
                                  locations='final_tracts',
                                  featureidkey='properties.BoroCT2010',
                                  color='Median Household Income',
                                  color_continuous_scale="Viridis",
                                  range_color=(0, 250005),
                                  hover_data=['Median Household Income'],
                                  center = {'lat': 40.79, 'lon': -73.99},
                                  zoom=10.5,
                                  mapbox_style='carto-positron')

fig_mapbox_2019.update_layout(height=700)
fig_mapbox_2019.show()

When looking at both the maps, at first glance it may not seem like much has changed, however that couldn't be further from the truth. In the latter map, the 2019 one, wealth is spread out a lot more throughout the city, and seemingly less concentrated, even spreading into the lower-east side, overall it seems that Manhattan generally has improved it's economic wellbeing over the past decade. The only concering area is northern manhattan around Harlem, where the wealth is seemingly not being spread, and very little seems to have been done to move that area, or break the mold there.

Overall, it seems that after this analysis, NYC is on the right track in Manhattan, and seems to be ensuring poverty concentration is being worked on, slowly but surely. It's nice to see tha programs like MIH (Mandatory inclusionary housing) seem to be working when just looking at the incomes, but at the end of the day, it's safe to say that Manhattan is improving fiscally, as both the richest and the poorest residents have seen large jumps in their household's income over the past decade. 